# Load dataset

In [ ]:
import pandas as pd
import numpy as np
import zipfile

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# dataset form: https://forms.gle/v531DJ2p37MmjMPG7

In [21]:
with zipfile.ZipFile('/content/drive/MyDrive/IA/2022-1/trab1-knn-chatbot/Treinamento_de_Chatbot.csv.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/drive/MyDrive/IA/2022-1/trab1-knn-chatbot/')

In [92]:
form = pd.read_csv('/content/drive/MyDrive/IA/2022-1/trab1-knn-chatbot/Treinamento_de_Chatbot.csv')

In [93]:
form

,Carimbo de data/hora,Pedir o cardápio,Adicionar um item ao pedido,Pedir a conta,Cancelar o pedido,Forma de pagamento,Acompanhar status do pedido,Inserir endereço para entrega
0,22/03/2022 14:51:28,me ve o cardapio,vou querer um xbacon,qual o total?,quero cancelar,no dinheiro,onde o pedido ta?,"rua barao do rio branco, 772"
1,22/03/2022 17:33:21,tem cardapio?,quero uma coca 600,quanto ficou?,cancelar pedido,pix,o pedido ta a caminho?,"avenida tal, bairro ministro, 345"
2,22/03/2022 17:36:19,o que tem no menu?,pizza de calabresa,me ve a conta,nao quero mais,vou pagar no crédito,o entregador já saiu?,"rua joao arinos, bairro jardim noroeste, 3457"
3,22/03/2022 18:38:39,o que ta servindo hoje?,um guaraná 2l,quanto da?,cancela,no debito,localizacao entregador,"Rua Gonçalo de Carvalho, 21"


In [94]:
form = form.drop(columns='Carimbo de data/hora')

In [95]:
form.columns

Index(['Pedir o cardápio', 'Adicionar um item ao pedido', 'Pedir a conta',
       'Cancelar o pedido', 'Forma de pagamento',
       'Acompanhar status do pedido', 'Inserir endereço para entrega'],
      dtype='object')

In [96]:
intentions = form.columns.to_list()
intentions

['Pedir o cardápio',
 'Adicionar um item ao pedido',
 'Pedir a conta',
 'Cancelar o pedido',
 'Forma de pagamento',
 'Acompanhar status do pedido',
 'Inserir endereço para entrega']

In [109]:
x = []
y = []

for intention in intentions:
  intention_values = form[intention].values.tolist()
  for intention_value in intention_values:
    x.append(intention_value)
    y.append(intention)

for i, j in zip(x, y):
  print(f'VALUE: {i} | LABEL: {j}')

VALUE: me ve o cardapio | LABEL: Pedir o cardápio
VALUE: tem cardapio? | LABEL: Pedir o cardápio
VALUE: o que tem no menu? | LABEL: Pedir o cardápio
VALUE: o que ta servindo hoje? | LABEL: Pedir o cardápio
VALUE: vou querer um xbacon | LABEL: Adicionar um item ao pedido
VALUE: quero uma coca 600 | LABEL: Adicionar um item ao pedido
VALUE: pizza de calabresa | LABEL: Adicionar um item ao pedido
VALUE: um guaraná 2l | LABEL: Adicionar um item ao pedido
VALUE: qual o total? | LABEL: Pedir a conta
VALUE: quanto ficou? | LABEL: Pedir a conta
VALUE: me ve a conta | LABEL: Pedir a conta
VALUE: quanto da? | LABEL: Pedir a conta
VALUE: quero cancelar | LABEL: Cancelar o pedido
VALUE: cancelar pedido | LABEL: Cancelar o pedido
VALUE: nao quero mais | LABEL: Cancelar o pedido
VALUE: cancela | LABEL: Cancelar o pedido
VALUE: no dinheiro | LABEL: Forma de pagamento
VALUE: pix | LABEL: Forma de pagamento
VALUE: vou pagar no crédito | LABEL: Forma de pagamento
VALUE: no debito | LABEL: Forma de pagam

# Split dataset in train, validation and test (75%, 12.5%, 12.5%)

In [110]:
import sklearn.model_selection as model_selection

In [129]:
xtrain, xtestval, ytrain, ytestval = model_selection.train_test_split(x, y, test_size=0.25, shuffle=True, stratify=y)

In [130]:
xtest, xval, ytest, yval = model_selection.train_test_split(xtestval, ytestval, test_size=0.50, shuffle=True)

In [131]:
print(f'X lenght: {len(x)} (Proportion: {len(x)/len(x)})')
print(f'Train lenght: {len(xtrain)} (Proportion: {len(xtrain)/len(x)})')
print(f'Validation lenght: {len(xval)} (Proportion: {len(xval)/len(x)})')
print(f'Test lenght: {len(xtest)} (Proportion: {len(xtest)/len(x)})')

X lenght: 28 (Proportion: 1.0)
Train lenght: 21 (Proportion: 0.75)
Validation lenght: 4 (Proportion: 0.14285714285714285)
Test lenght: 3 (Proportion: 0.10714285714285714)


In [132]:
xtrain

['o pedido ta a caminho?',
 'cancela',
 'qual o total?',
 'Rua Gonçalo de Carvalho, 21',
 'quanto ficou?',
 'o que tem no menu?',
 'o entregador já saiu?',
 'um guaraná 2l',
 'cancelar pedido',
 'no dinheiro',
 'quero cancelar',
 'pizza de calabresa',
 'me ve a conta',
 'rua barao do rio branco, 772',
 'no debito',
 'quero uma coca 600',
 'localizacao entregador',
 'rua joao arinos, bairro jardim noroeste, 3457',
 'pix',
 'me ve o cardapio',
 'tem cardapio?']

In [133]:
xval

['quanto da?',
 'vou querer um xbacon',
 'avenida tal, bairro ministro, 345',
 'nao quero mais']

In [134]:
xtest

['o que ta servindo hoje?', 'onde o pedido ta?', 'vou pagar no crédito']

# Pré-process dataset

In [ ]:
!pip install -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
converter = SentenceTransformer('multi-qa-distilbert-cos-v1')

In [135]:
xtrain_embeddings = converter.encode(xtrain)
xval_embeddings = converter.encode(xval)
xtest_embeddings = converter.encode(xtest)

In [138]:
print(xtrain_embeddings.shape)
print(xval_embeddings.shape)
print(xtest_embeddings.shape)

(21, 768)
(4, 768)
(3, 768)


# Model induction

In [140]:
import sklearn.neighbors as neighbors

In [159]:
classifier = neighbors.KNeighborsClassifier(n_neighbors=3)

In [160]:
model = classifier.fit(xtrain_embeddings, ytrain)

In [161]:
for xval_embedding, y in zip(xval_embeddings, yval):
  print(f'{model.predict([xval_embedding])} | {y}')

['Pedir a conta'] | Pedir a conta
['Acompanhar status do pedido'] | Adicionar um item ao pedido
['Acompanhar status do pedido'] | Inserir endereço para entrega
['Acompanhar status do pedido'] | Cancelar o pedido
